In [2]:
from keras.models import Model
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, Lambda, MaxPooling2D, Reshape
from keras.layers import Dropout
from keras import regularizers
from keras.engine.topology import Input
from keras.optimizers import Adam
from keras import backend as K
#import numpy as np

In [7]:
from Prepare_data import Get_data
from utils import to_file_params, TimeControll, data_generator

In [8]:
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import json
from os.path import exists
from os import makedirs

In [34]:
#MAIN
TYPE = 0

#OTHERS
DESCRIPTION = "Testing generator"
PRINT_LOGS = False
SHOW_FIGURES = True

#IMAGES
RESOLUTION = (96,96)
GRAYSCALE = True
INPUT_SHAPE = (RESOLUTION[0], RESOLUTION[1], 1 if GRAYSCALE else 3)
NUM_EXAMPLES = 15 if TYPE != 0 else 15
START_EXAMPLES = 0 if TYPE != 0 else 0

#FOLDERS
NUM_FOLDERS = 40 if TYPE != 0 else 5
START_FOLDER = 0 if TYPE != 0 else 0
BATCH_SIZE_FOLDER = 2 if TYPE != 0 else 2

#FOLDERS VALIDATION
NUM_FOLDERS_VAL = 20 if TYPE != 0 else 2
START_FOLDER_VAL = 900 if TYPE != 0 else 900
BATCH_SIZE_FOLDER_VAL = 2 if TYPE != 0 else 2


#TRAIN ITTERATION
#BATCH_SIZE = 80 if TYPE != 0 else 50
EPOCHS = 20 if TYPE != 0 else 3

#TRAIN
LEARNING_RATE = 0.001
L2 = 0.0007

#DIRECTIONS
NAME_HISTORY = f"history/FaceModel_whaleType/train/history{TYPE}.txt" if TYPE != 0 else f"history/FaceModel_whaleType/train/history_train_test{TYPE}.txt"
NAME_JSON_HISTORY = f"history/FaceModel_whaleType/JSON/history{TYPE}.json" if TYPE != 0 else f"history/FaceModel_whaleType/JSON/history_test{TYPE}.json"

NAME_FIGURE = f"figures/FaceModel_whaleType/train/{TYPE}/"

SAVE_MODEL_DIR = "models/FaceModel_whaleType"
SAVE_EACH_MODEL_DIR = f"{SAVE_MODEL_DIR}/{TYPE}"

SAVE_MODEL_NAME = f"{SAVE_MODEL_DIR}/Model_{TYPE}.h5" if TYPE != 0 else f"{SAVE_MODEL_DIR}/Model_test_{TYPE}.h5" 
SAVE_EACH_MODEL_NAME = (f"{SAVE_EACH_MODEL_DIR}/Model_" + "{epoch:02d}-{val_loss:.2f}.h5") if TYPE != 0 else (f"{SAVE_EACH_MODEL_DIR}/Model_test_" + "{epoch:02d}-{val_loss:.2f}.h5")

DATASET_P = DATASET_N = "../Datasets/Faces_dataset/Faces"

In [35]:
TRAIN_EXAMPLES = NUM_EXAMPLES*NUM_FOLDERS*2
TEST_EXAMPLES = NUM_EXAMPLES*NUM_FOLDERS_VAL*2
ALL_EXAMPLES = TRAIN_EXAMPLES+TEST_EXAMPLES

In [36]:
trainGen = data_generator(DATASET_P,
                          DATASET_N,
                          resolution = RESOLUTION,
                          grayscale = GRAYSCALE,
                          num_examples = NUM_EXAMPLES,
                          start_examples = START_EXAMPLES,
                          num_folders = NUM_FOLDERS,
                          start_folder = START_FOLDER,
                          batch_size_folder = BATCH_SIZE_FOLDER,
                          input_shape = INPUT_SHAPE,
                          print_logs = PRINT_LOGS)

In [37]:
valGen = data_generator(DATASET_P,
                        DATASET_N,
                        resolution = RESOLUTION,
                        grayscale = GRAYSCALE,
                        num_examples = NUM_EXAMPLES,
                        start_examples = START_EXAMPLES,
                        num_folders = NUM_FOLDERS_VAL,
                        start_folder = START_FOLDER_VAL,
                        batch_size_folder = BATCH_SIZE_FOLDER_VAL,
                        input_shape = INPUT_SHAPE,
                        print_logs = PRINT_LOGS)

In [13]:
def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y) # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y) # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y) # no activation # Restore the number of original features
    y = Add()([x,y]) # Add the bypass connection
    y = Activation('relu')(y)
    return y

In [14]:
def build_model(img_shape, lr, l2, activation='sigmoid'):

    ##############
    # BRANCH MODEL
    ##############
    regul  = regularizers.l2(l2)
    optim  = Adam(lr=lr)
    kwargs = {'padding':'same', 'kernel_regularizer':regul}

    inp = Input(shape=img_shape) # 384x384x1
    x   = Conv2D(64, (9,9), strides=2, activation='relu', **kwargs)(inp)

    x   = MaxPooling2D((2, 2), strides=(2, 2))(x) # 96x96x64
    for _ in range(1):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3,3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1,1), activation='relu', **kwargs)(x) # 48x48x128
    for _ in range(2): x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1,1), activation='relu', **kwargs)(x) # 24x24x256
    for _ in range(2): x = subblock(x, 64, **kwargs)
    x = Dropout(0.25)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1,1), activation='relu', **kwargs)(x) # 12x12x384
    for _ in range(2): x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x) # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1,1), activation='relu', **kwargs)(x) # 6x6x512
    for _ in range(2): x = subblock(x, 128, **kwargs)
    
    x             = GlobalMaxPooling2D()(x) # 512
    branch_model  = Model(inp, x)
    
    ############
    # HEAD MODEL
    ############
    mid        = 32
    xa_inp     = Input(shape=branch_model.output_shape[1:])
    xb_inp     = Input(shape=branch_model.output_shape[1:])
    x1         = Lambda(lambda x : x[0]*x[1])([xa_inp, xb_inp])
    x2         = Lambda(lambda x : x[0] + x[1])([xa_inp, xb_inp])
    x3         = Lambda(lambda x : K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4         = Lambda(lambda x : K.square(x))(x3)
    x          = Concatenate()([x1, x2, x3, x4])

    x          = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x          = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x          = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x          = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x          = Flatten(name='flatten')(x)
    
    # Weighted sum implemented as a Dense layer.
    x          = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    # Complete model is constructed by calling the branch model on each input image,
    # and then the head model on the resulting 512-vectors.
    img_a      = Input(shape=img_shape)
    img_b      = Input(shape=img_shape)
    xa         = branch_model(img_a)
    xb         = branch_model(img_b)
    x          = head_model([xa, xb])
    model      = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
    return model, branch_model, head_model

In [27]:
FaceModel, _,_ = build_model(INPUT_SHAPE, lr = LEARNING_RATE, l2 = L2)


In [28]:
to_file_params(NAME_HISTORY, [f"input_shape = {INPUT_SHAPE}\nnum_examples = {NUM_EXAMPLES}\nnum_folders = {NUM_FOLDERS}\nstart_folder = {START_FOLDER}\nbatch_size_folder = {BATCH_SIZE_FOLDER}\nnum_folders_val = {NUM_FOLDERS_VAL}\nstart_folder_val = {START_FOLDER_VAL}\nbatch_size_folder_val = {BATCH_SIZE_FOLDER_VAL}\nepochs = {EPOCHS}\nlearning_rate = {LEARNING_RATE}\nl2 = {L2}\nname_history = {NAME_HISTORY}\nsave_model_name = {SAVE_MODEL_NAME}\ndataset_positive = {DATASET_P}\ndataset_negative = {DATASET_N}\ndescription = {DESCRIPTION}\n"], with_lines = False)

to_file_params(NAME_HISTORY, [f"all_examples = {ALL_EXAMPLES}\ntrain_examples = {TRAIN_EXAMPLES}\ntest_examples = {TEST_EXAMPLES}\n"], with_lines = False)

In [29]:
if not exists(SAVE_EACH_MODEL_DIR):
    makedirs(SAVE_EACH_MODEL_DIR)

In [30]:
callbacks = ModelCheckpoint(SAVE_EACH_MODEL_NAME, 
                            monitor=["val_loss"], 
                            mode="min",
                            verbose = 1)

In [31]:
callback_list = [callbacks]

In [32]:
my_time = TimeControll()
time_start = my_time.get_start_time()
to_file_params(NAME_HISTORY, [f"\tStart time = {time_start[0]}:{time_start[1]}"], with_lines = False)

In [38]:
hist = FaceModel.fit_generator(trainGen, 
                               epochs = EPOCHS, 
                               steps_per_epoch = NUM_FOLDERS//BATCH_SIZE_FOLDER,
                               validation_data = valGen,
                               validation_steps = NUM_FOLDERS_VAL//BATCH_SIZE_FOLDER_VAL,
                               callbacks = callback_list,
                               verbose = 1)

Epoch 1/3
2/2 [==============================] - 32s 16s/step - loss: 3.4469 - binary_crossentropy: 0.6835 - acc: 0.6000 - val_loss: 3.5236 - val_binary_crossentropy: 0.7762 - val_acc: 0.5000

Epoch 00001: saving model to models/FaceModel_whaleType/0/Model_test_01-3.52.h5
Epoch 2/3
2/2 [==============================] - 1s 479ms/step - loss: 3.4237 - binary_crossentropy: 0.6822 - acc: 0.5083 - val_loss: 3.3989 - val_binary_crossentropy: 0.6750 - val_acc: 0.5333

Epoch 00002: saving model to models/FaceModel_whaleType/0/Model_test_02-3.40.h5
Epoch 3/3
2/2 [==============================] - 1s 472ms/step - loss: 3.4127 - binary_crossentropy: 0.6946 - acc: 0.5250 - val_loss: 3.3706 - val_binary_crossentropy: 0.6698 - val_acc: 0.5667

Epoch 00003: saving model to models/FaceModel_whaleType/0/Model_test_03-3.37.h5


In [39]:
my_time.set_end_time()

In [40]:
FaceModel.save(SAVE_MODEL_NAME)

In [41]:
time_end = my_time.get_end_time()
time_spend = my_time.get_spend_time()

In [42]:
to_file_params(NAME_HISTORY,[f"\tEnd time = {time_end[0]}:{time_end[1]}\n\tSpend time = {time_spend[0]}:{time_spend[1]}:{time_spend[2]}"], with_lines = False)

to_file_params(NAME_HISTORY,[f"\tAccuracy: {hist.history['acc'][-1]}, Val Accuracy: {hist.history['val_acc'][-1]}, Loss: {hist.history['loss'][-1]}, Val Loss: {hist.history['val_loss'][-1]}"], with_lines = False)


In [43]:
out = [f"Epoch {i+1}/{EPOCHS}\n\t\t loss: {loss}, acc: {acc}, val_loss: {val_loss}, val_acc: {val_acc}" for i,(loss,acc,val_loss,val_acc) in enumerate(zip(hist.history["loss"],hist.history["acc"],hist.history["val_loss"],hist.history["val_acc"]))]

to_file_params(NAME_HISTORY, out)

# Testing history in json format

In [34]:
history_dict = hist.history

In [35]:
json.dump(history_dict, open(NAME_JSON_HISTORY, 'a'))

In [38]:
#history_dict["acc"]

[0.5033333321412404,
 0.5344444493452708,
 0.5688888887564342,
 0.5133333325386047,
 0.5666666666666667,
 0.6199999968210856,
 0.6755555510520935,
 0.746666673819224,
 0.751111110051473,
 0.7522222240765889,
 0.7955555597941081,
 0.8022222240765889,
 0.8544444441795349,
 0.896666657924652,
 0.950000003973643,
 0.9711111028989156,
 0.9699999968210856,
 0.9355555534362793,
 0.9377777695655822,
 0.9377777695655822]

In [39]:
with open(NAME_JSON_HISTORY, 'r') as fl:
    json_file = json.load(fl)

In [40]:
#json_file["acc"]

[0.5033333321412404,
 0.5344444493452708,
 0.5688888887564342,
 0.5133333325386047,
 0.5666666666666667,
 0.6199999968210856,
 0.6755555510520935,
 0.746666673819224,
 0.751111110051473,
 0.7522222240765889,
 0.7955555597941081,
 0.8022222240765889,
 0.8544444441795349,
 0.896666657924652,
 0.950000003973643,
 0.9711111028989156,
 0.9699999968210856,
 0.9355555534362793,
 0.9377777695655822,
 0.9377777695655822]

# Plots creation

In [41]:
def monitor_process(history, metrics):
    metrics_list = {m:history[m] for m in metrics}
    epochs = range(len(metrics_list[metrics[0]]))
    
    if not exists(NAME_FIGURE):
        makedirs(NAME_FIGURE)
    
    for metric,values in metrics_list.items():
        plt.plot(epochs, values)
        plt.title(f"Type: {TYPE} - {metric}")


        if TYPE != 0:
            plt.savefig(NAME_FIGURE + f'{metric}-{values[-1]:.2f}.jpg', quality = 100, dpi = 150)
        else:
            plt.savefig(NAME_FIGURE + f'test_{metric}-{values[-1]:.2f}.jpg', quality = 100, dpi = 150)
        
        plt.show()

        #plt.savefig("history/test_accuracy.jpg", bbox_inches = "tight", quality = 100, dpi = 150)#save_fig


In [58]:
if SHOW_FIGURES:
    from utils import monitor_process
    
    monitor_process(hist.history, ["acc","val_acc","loss","val_loss"], NAME_FIGURE, TYPE)

ImportError: cannot import name 'monitor_process'

# Save model in json format

In [88]:
model_json = FaceModel.to_json()

In [89]:
with open("test_history/json_model.json", 'w') as json_file:
    json_file.write(model_json)